In [3]:
import praw
import pandas as pd
import numpy as np
import emoji
import unicodedata

from transformers import AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
import spacy
import torch
import matplotlib.pyplot as plt

from tqdm import tqdm


reddit = praw.Reddit("learning")
header = {"User-Agent": "my-bot/0.0.1 by u/Karmz0a"}

add demo mode that only uses 15 posts
Input String + Search Subreddits
filter out meme subs, make sure theyre relevant, check for satire
search directly for comments, not posts -> comments

In [4]:
topic = input()
demo = False

subreddit = reddit.subreddit(topic)
count = 10 if demo else 10 if subreddit.subscribers > 10000000 else 100
non_sticky = []
i = 0
for submission in subreddit.hot(limit = 125):
    if submission.stickied or submission.num_comments == 0:
        continue
    else:
        i = i + 1
        non_sticky.append(submission)
        if i == count:
            break

data = []
for submission in non_sticky:
    
    submission.comments.replace_more(limit = None)
    commentList = submission.comments.list()
    for comment in commentList:
        data.append({
            "author": comment.author,
            "original post" : submission.id,
            "upvotes": comment.score,
            "body": emoji.demojize(comment.body, delimiters=(":", ":")).replace("\n", " ").replace("\r", " "),
        })

Data Cleaning

In [5]:
commentFrame = pd.DataFrame(data)

commentFrame = commentFrame[(commentFrame["author"] != "AutoModerator") & 
                            (commentFrame["author"].notna()) &
                            (commentFrame["body"] != "[removed]")
                            ].drop_duplicates()

commentFrame["body"] = commentFrame["body"].apply(lambda x: unicodedata.normalize("NFC", x ))

commentFrame["upvoteScale"] = np.where(
    commentFrame["upvotes"] == 0,
    0,
    np.log10(commentFrame["upvotes"] + 1)
)


C:\Users\pweir\AppData\Roaming\Python\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\pweir\AppData\Roaming\Python\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Sentiment Analysis
give user a list of spaCy labels to choose what is relevant and what is not

filter SPACY keywords -> work of art, product, org person etc

In [6]:
nlp = spacy.load("en_core_web_sm", disable= ["parser", "tagger"])
embedder = SentenceTransformer("all-MiniLM-L6-v2")

spacy_entities = ["ORG", "PRODUCT", "PERSON", "WORK_OF_ART"] # make this multiple choice for the user
entity = []
for doc in nlp.pipe(commentFrame["body"], batch_size= 100, n_process=-1):
    for ent in doc.ents:
        entity.append((ent.text, ent.label_))


df_entities = pd.DataFrame(entity, columns=["entity", "label"])
e = df_entities[df_entities["label"].isin(spacy_entities)].value_counts().head(18)
top_ent = [idx[0] for idx in e.index]


comments = commentFrame["body"].tolist()
query_emb =  embedder.encode(top_ent, normalize_embeddings= True)

comment_emb = embedder.encode(comments, normalize_embeddings= True, show_progress_bar= True, batch_size= 16)

relevance = embedder.similarity(query_emb, comment_emb)
best_score = torch.max(relevance, dim = 0).values.tolist()

relevance = embedder.similarity(query_emb, comment_emb)
best_score = torch.max(relevance, dim = 0).values.tolist()
commentFrame["relevance"] =  best_score

cutoff = np.percentile(commentFrame["relevance"], 10)
commentFrame = commentFrame[commentFrame["relevance"] > cutoff]

Batches:   0%|          | 0/677 [00:00<?, ?it/s]

In [7]:
comments = commentFrame["body"].tolist()
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("sentiment-analysis", model = "AG6019/reddit-comment-sentiment-final", device = device)

label_map = {"LABEL_0": "NEGATIVE", "LABEL_1": "POSITIVE"}

batch_size = 8
results = []

for i in tqdm(range(0, len(comments), batch_size), desc = "sentiment analysis"):
    batch = comments[i:i+batch_size]
    sentiment = classifier(batch, truncation = True, max_length = 128)
    results.extend(sentiment)

commentFrame["sentimentLabel"] = ["NEUTRAL" if r["score"] < 0.85 else label_map[r["label"]] for r in results]
commentFrame["sentimentScore"] = [r["score"] for r in results]

print(commentFrame.head())

Device set to use cpu
sentiment analysis: 100%|██████████| 1218/1218 [16:26<00:00,  1.23it/s]

                author original post  upvotes  \
0         linkling1039       1nk7ow2        1   
1              gomster       1nk7ow2        1   
2          Jafranci715       1nk7ow2        1   
3    Salt-Analysis1319       1nk7ow2        1   
4  A-Centrifugal-Force       1nk7ow2        1   

                                                body  upvoteScale  relevance  \
0  I'm really jealous of the people that got to p...      0.30103   0.159744   
1                It’s crazy how well Galaxy holds up      0.30103   0.175538   
2       Does the switch 2 version launch on off 2nd?      0.30103   0.620313   
3  As someone who hasn't played either game since...      0.30103   0.547247   
4  They look so good! Can’t wait to play both of ...      0.30103   0.188442   

  sentimentLabel  sentimentScore  
0       POSITIVE        0.989360  
1       NEGATIVE        0.974828  
2        NEUTRAL        0.518872  
3        NEUTRAL        0.573924  
4       POSITIVE        0.993539  


Account for Sarcasm and Upvotes

In [9]:
print(commentFrame["sentimentLabel"].value_counts())
commentFrame.to_pickle("comments.pkl")

sentimentLabel
POSITIVE    5320
NEUTRAL     2783
NEGATIVE    1639
Name: count, dtype: int64


sentiment assortivity
overall sentiment of the subreddit- battle ground or agreement